In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://www.scrapethissite.com/pages/forms/'

In [5]:
page = requests.get(url)

In [7]:
soup = BeautifulSoup(page.text, 'html')

In [13]:
# used to find all instances but can't extract using .text
soup.find_all('p', class_ = 'lead')

[<p class="lead">
                             Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.
                             Take a look at how pagination and search elements change the URL as your browse. Build a web scraper that can conduct searches and paginate through the results.
                         </p>]

In [16]:
# this one can finde the text
soup.find('p', class_ = 'lead').text.strip()

'Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.\n                            Take a look at how pagination and search elements change the URL as your browse. Build a web scraper that can conduct searches and paginate through the results.'

In [17]:
# used to find all instances but can't extract using .text
soup.find_all('p', class_ = 'lead')

[<p class="lead">
                             Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.
                             Take a look at how pagination and search elements change the URL as your browse. Build a web scraper that can conduct searches and paginate through the results.
                         </p>]

# Intento 2

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

def scroll_to_button(driver, button):
    driver.execute_script("arguments[0].scrollIntoView();", button)

chrome_options = Options()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
# wait = WebDriverWait(driver, 20)

url = 'https://es.wallapop.com/app/search?filters_source=search_box&keywords=retroid%20pocket%205'
driver.get(url)

try:
    cookie_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler")))
    cookie_button.click()
except TimeoutException:
    print("No se encontró el botón de cookies o no apareció el diálogo.")


def click_ver_mas():
    try:
        # Esperar hasta que el botón 'Ver más' esté presente en el DOM
        button = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//walla-button[@id='btn-load-more']"))
        )
        # Desplazar la página hasta el botón para asegurarse de que es visible
        driver.execute_script('arguments[0].scrollIntoView({behavior: "auto", block: "center"});', button)
        time.sleep(1)
        # Acceder al Shadow DOM del botón
        shadow_root = driver.execute_script('return arguments[0].shadowRoot', button)
        # Encontrar el botón dentro del Shadow DOM y hacer clic
        shadow_button = WebDriverWait(shadow_root, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.walla-button__button--medium"))
        )
        shadow_button.click()
        time.sleep(2)
        return True
    except Exception as e:
        print(f"Error al hacer clic en 'Ver más'")
        return False

# Hacer clic en 'Ver más' hasta que no sea posible
while click_ver_mas():
    pass

print("Se han cargado todos los elementos disponibles.")

# Esperar a que se carguen los elementos
WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "ItemCardList__item")))

# Obtener el HTML y parsearlo
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Encontrar y procesar los elementos
items = soup.find_all(class_="ItemCardList__item")
for item in items:
    title = item.find(class_="ItemCard__title")
    price = item.find(class_="ItemCard__price")
    if title and price:
        print(f"Título: {title.text.strip()}, Precio: {price.text.strip()}")

driver.quit()

Error al hacer clic en 'Ver más': Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6E916C6A5+28789]
	(No symbol) [0x00007FF6E90D5B20]
	(No symbol) [0x00007FF6E8F68F9A]
	(No symbol) [0x00007FF6E8FBF346]
	(No symbol) [0x00007FF6E8FBF57C]
	(No symbol) [0x00007FF6E9012B17]
	(No symbol) [0x00007FF6E8FE736F]
	(No symbol) [0x00007FF6E900F7E3]
	(No symbol) [0x00007FF6E8FE7103]
	(No symbol) [0x00007FF6E8FAFFC0]
	(No symbol) [0x00007FF6E8FB1273]
	GetHandleVerifier [0x00007FF6E94B1AED+3458237]
	GetHandleVerifier [0x00007FF6E94C829C+3550316]
	GetHandleVerifier [0x00007FF6E94BDB9D+3507565]
	GetHandleVerifier [0x00007FF6E9232C6A+841274]
	(No symbol) [0x00007FF6E90E09EF]
	(No symbol) [0x00007FF6E90DCB34]
	(No symbol) [0x00007FF6E90DCCD6]
	(No symbol) [0x00007FF6E90CC119]
	BaseThreadInitThunk [0x00007FFD5C3D259D+29]
	RtlUserThreadStart [0x00007FFD5E08AF38+40]

Se han cargado todos los elementos disponibles.
Título: Vendo Retroid Pocket 5, Precio: 240,00 €
Título: Retroid Pocket 5 8-128, Precio: 230,

In [124]:
for item in soup.find_all(class_="ItemCardList__item"):

    price = item.find('span', class_="ItemCard__price").text.strip()
    name = item.get('title')
    webpage = item.get('href')

    print(name, price, webpage)

Retroid pocket 4 Pro 160,00 € https://es.wallapop.com/item/retroid-pocket-4-pro-1107098476
Vendo Retroid Pocket 5  240,00 € https://es.wallapop.com/item/vendo-retroid-pocket-5-1107127250
Retroid Pocket 5 vs Nintendo Switch PS5 XBox Ps4 249,00 € https://es.wallapop.com/item/retroid-pocket-5-vs-nintendo-switch-ps5-xbox-ps4-1106735038
Retroid Pocket 4 Pro 165,00 € https://es.wallapop.com/item/retroid-pocket-4-pro-1105385055
Retroid pocket 4 pro  0,00 € https://es.wallapop.com/item/retroid-pocket-4-pro-1105077835
Retroid Pocket Mini 205,00 € https://es.wallapop.com/item/retroid-pocket-mini-1105998164
Retroid Pocket 4 Pro 128gb 220,00 € https://es.wallapop.com/item/retroid-pocket-4-pro-128gb-1103658233
16GB 1TB AYANEO Pocket S vs Nintendo Switch Ps5 599,00 € https://es.wallapop.com/item/16gb-1tb-ayaneo-pocket-s-vs-nintendo-switch-ps5-1106738184
12GB 128GB AYANEO Pocket S vs Nintendo Switch Ps5 499,00 € https://es.wallapop.com/item/12gb-128gb-ayaneo-pocket-s-vs-nintendo-switch-ps5-1106737292